In [48]:
# Import the modules and dependencies
# Contrary to usual practice, some of the modules are imported in the cells where they are used, per the starter code.
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

# Create filter for deprecation warnings
import warnings
warnings.filterwarnings('ignore')


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [50]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path("Resources/lending_data.csv")
)

# Review the DataFrame
lending_data_df.head() # default is 5 rows


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [51]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features: Using uppercase X to follow the scikit-learn convention for features
X = lending_data_df.drop(columns=['loan_status']) # Note: axis=1 is the default for drop(), so it is not needed here

In [68]:
# Review the y variable Series

# Convert y Series to DataFrame for better display
y_df = pd.DataFrame(y, columns=['loan_status'])

# Show head of y DataFrame
y_df.head()

,loan_status
0,0
1,0
2,0
3,0
4,0


In [53]:
# Review the X variable DataFrame
X.head(5) # default is 5 rows

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [54]:
# Check the balance of our target values
y.value_counts() # for loan_status

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [55]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
# Note: stratify=y ensures that the training and test datasets have the same proportions of classes as the original dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [56]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train) # Note: the model is fitted to the training data only

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [69]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test) # Note: the model is predicting using the testing data only

# Print the number of rows in the test dataset
print(f"Test Dataset Rows: {len(X_test)}")

# Print the number of predictions
print(f"Predictions: {len(testing_predictions)}") 

Test Dataset Rows: 19384
Predictions: 19384


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [58]:
# Print the balanced_accuracy score of the model
# Note: balanced_accuracy_score() is used for classification problems, not regression problems
logistic_acc_score = balanced_accuracy_score(y_test, testing_predictions)
print(f"Balanced Accuracy Score: {logistic_acc_score}")

Balanced Accuracy Score: 0.9442676901753825


In [70]:
# Generate a confusion matrix for the model
logistic_matrix = confusion_matrix(y_test, testing_predictions) 

# Convert the confusion matrix to a DataFrame
confusion_df = pd.DataFrame(
    logistic_matrix, 
    index=["Actual 0", "Actual 1"], 
    columns=["Predicted 0", "Predicted 1"]
)

print("Confusion Matrix:")
print(confusion_df)

Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0        18679           80
Actual 1           67          558


In [60]:
# Print the classification report for the model

# Mapping numerical labels to string labels for better readability
label_dict = {0: "Healthy Loan", 1: "High-Risk Loan"}

# Convert numerical labels to string labels for y_test and testing_predictions
y_test_str = y_test.map(label_dict)
testing_predictions_str = pd.Series(testing_predictions).map(label_dict)

# Generate and print the classification report
logistic_report = classification_report(y_test_str, testing_predictions_str, target_names=["Healthy Loan", "High-Risk Loan"])
print(logistic_report)


                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (`Healthy Loan`) and `1` (`High-Risk Loan`) labels?

**Answer:** The model appears to perform extremely well for data associated with the `Healthy Loan` label (value `0`), with suspiciously perfect or near-perfect metrics. For data with the `High-Risk Loan` label (value `1`), the model doesn't perform as infallibily as for the `Healthy Loan` cohort, but it's still correct at classification nearly 90% of the time. The accuracy, or the simple ratio of correctly predicted instances to the total instances in the dataset, is an impressive 99%.

These results must be interpreted with caution, given that the original dataset was highly imbalanced (75,036 `Healthy Loan`s vs 2,500 `High-Risk Loan`s). Despite use of balanced accuracy and stratified sampling, techniques specifically designed to handle imbalanced datasets should be considered.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [61]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model

# # Assign a random_state parameter of 1 to the model
rand_param = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = rand_param.fit_resample(X_train, y_train) # Note: the model is fitted to the training data only

In [62]:
# Count the distinct values of the resampled labels data
y_res.value_counts() # The volume of the minority class has been increased to match the majority class by artificially oversampling

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [63]:
# Instantiate the Logistic Regression model

# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1) # lbfgs is the default solver

# Fit the model using the resampled training data
classifier.fit(X_res, y_res) # Note: the model is fitted to the training data only

# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [64]:
# Print the balanced_accuracy score of the model 
predicted_logistic_acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Balanced Accuracy Score: {predicted_logistic_acc_score}")

Balanced Accuracy Score: 0.9959744975744975


In [71]:
# Generate a confusion matrix for the model
predicted_logistic_matrix = confusion_matrix(y_test, predictions)

# Convert the confusion matrix to a DataFrame
confusion2_df = pd.DataFrame(
    predicted_logistic_matrix, 
    index=["Actual 0", "Actual 1"], 
    columns=["Predicted 0", "Predicted 1"]
)

print("Confusion Matrix:")
print(confusion2_df)

Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0        18668           91
Actual 1            2          623


In [66]:
# Print the classification report for the model

# Mapping numerical labels to string labels for better readability
label_dict = {0: "Healthy Loan", 1: "High-Risk Loan"}

# Convert numerical labels to string labels for y_test and predictions
y_test_str = y_test.map(label_dict)
predictions_str = pd.Series(predictions).map(label_dict)

# Generate and print the classification report
predicted_logistic_classreport = classification_report(y_test_str, predictions_str, target_names=["Healthy Loan", "High-Risk Loan"])
print(predicted_logistic_classreport)


                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      1.00      0.93       625

      accuracy                           1.00     19384
     macro avg       0.94      1.00      0.96     19384
  weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (`Healthy Loan`) and `1` (`High-Risk Loan`) labels?

**Answer:** The model fit with oversampled data still performs exceptionally well on both the `Healthy Loan` (`0`) and `High-Risk Loan` (`1`) labels. For the `Healthy Loan` cohort, the model has a 100% precision rate and for the `High-Risk Loan` group, the precision is 87%. The finding is interesting because models trained on resampled data tend to perform better on the minority class, which indeed happened here, while often performing worse on the majority class, which did not happen in this case.

The recall still is 100% for `High-Risk Loan`s, indicating that the model is capturing almost all the high-risk cases. This may be indicative of an excellent model, but the results are almost "too good", and they might also be a warning sign of overfitting or data leakage. This can be investigated by examining performance metrics on a validation set that the model has not seen during training.